# 🤔Qualidade dos Dados

A análise da qualidade dos dados é fundamental para garantir a confiabilidade dos resultados obtidos nas consultas e análises de negócio. Nesta seção, serão abordados os principais aspectos relacionados à integridade, consistência e completude dos dados presentes nas tabelas do projeto.

**Principais pontos a serem avaliados:**
- **Valores nulos ou ausentes:** Identificar campos com alta incidência de valores nulos ou vazios, que podem impactar análises e decisões.
- **Duplicidade de registros:** Verificar a existência de registros duplicados, especialmente em tabelas de transações e clientes.
- **Consistência de tipos de dados:** Garantir que os campos estejam com os tipos de dados corretos (ex: datas, valores numéricos, categorias).
- **Validação de regras de negócio:** Conferir se os dados seguem as regras estabelecidas, como formatos de CEP válidos, valores positivos para vendas, etc.
- **Referencial entre tabelas:** Avaliar se as chaves estrangeiras estão corretamente relacionadas, evitando registros órfãos.

In [0]:
%sql
USE CATALOG `100cep_gateway`;

In [0]:
%sql
USE SCHEMA gold;

In [0]:
# QUALIDADE DA TABELA DIM_CHARGEBACKS

# 1. Valores nulos críticos na tabela dim_chargebacks
spark.sql("""
SELECT
  SUM(CASE WHEN chargeback_id IS NULL THEN 1 END) AS nulo_id_chargeback,
  SUM(CASE WHEN motivo_chargeback IS NULL THEN 1 END) AS nulo_motivo_chargeback,
  SUM(CASE WHEN status_chargeback IS NULL THEN 1 END) AS nulo_status_chargeback,
  SUM(CASE WHEN resposta_emitente IS NULL THEN 1 END) AS nulo_resposta_emitente,
  SUM(CASE WHEN resposta_adquirente IS NULL THEN 1 END) AS nulo_resposta_adquirente  
FROM dim_chargebacks;
""").display()

# 2. Verificando duplicidade de chargeback_id
spark.sql("""
SELECT
  chargeback_id,
  COUNT(*) AS qtd
FROM dim_chargebacks
GROUP BY chargeback_id
HAVING COUNT(*) > 1;
""").display()

# 3. Verificando valores distintos nas principais colunas
spark.sql("""
SELECT
  COUNT(*) AS total,
  COUNT(DISTINCT chargeback_id) AS total_distinct_chargeback_id,
  COUNT(DISTINCT motivo_chargeback) AS total_distinct_motivo_chargeback,
  COUNT(DISTINCT status_chargeback) AS total_distinct_status_chargeback
FROM dim_chargebacks;
""").display()

In [0]:
# QUALIDADE DA TABELA DIM_CLIENTES
# 1. Valores nulos críticos na tabela dim_clientes
spark.sql("""SELECT
  SUM(CASE WHEN cliente_id IS NULL THEN 1 END) AS nulo_id_cliente,
  SUM(CASE WHEN cliente_id_unico IS NULL THEN 1 END) AS nulo_id_cliente_unico
FROM dim_clientes;""").display()

# 2. Verificando se há duplicidades nos dados
spark.sql("""SELECT
  COUNT(*) AS total,
  COUNT(DISTINCT cliente_id) AS total_distinct_id,
  COUNT(DISTINCT cliente_id_unico) AS total_distinct_cliente_unico,
  COUNT(DISTINCT cep_prefixo) AS total_distinct_cep
FROM dim_clientes;""").display()

# 3. Verificando se todos os cep_prefixo possuem 5 números
spark.sql("""SELECT
  COUNT(cep_prefixo) AS total_cep_prefixo,
  COUNT(DISTINCT CASE WHEN LENGTH(cep_prefixo) != 5 THEN 1 END) AS total_invalidos
FROM dim_clientes;""").display()


In [0]:
# QUALIDADE DA TABELA DIM_DATA
# 1. Valores nulos críticos na tabela dim_data
spark.sql("""SELECT
  SUM(CASE WHEN data_calendario IS NULL THEN 1 END) AS nulo_data_calendario
FROM dim_data;""").display()

# 2. Se todas as datas na dim_data são representadas na fato_transacoes
spark.sql("""
SELECT
  dt.data_calendario,
  CASE WHEN ft.data_pedido IS NULL THEN 'NÃO REPRESENTADA' ELSE 'REPRESENTADA' END AS status_representacao
FROM dim_data AS dt
LEFT JOIN fato_transacoes AS ft
  ON dt.data_calendario = ft.data_pedido
GROUP BY dt.data_calendario, ft.data_pedido;
""").display()

# 3. Se todas as datas na fato_transacoes são representadas na dim_data 
spark.sql("""
SELECT
  ft.data_pedido ,
  CASE WHEN dt.data_calendario IS NULL THEN 'NÃO REPRESENTADA' ELSE 'REPRESENTADA' END AS status_representacao
FROM fato_transacoes AS ft
LEFT JOIN dim_data AS dt
  ON dt.data_calendario = ft.data_pedido
GROUP BY ft.data_pedido, dt.data_calendario;
""").display()

# 4. Verificando se há datas repetidas
spark.sql("""
SELECT
  data_calendario,
  COUNT(*) AS qtd
FROM dim_data
GROUP BY data_calendario
HAVING COUNT(*) > 1;
""").display()

In [0]:
# QUALIDADE DA TABELA DIM_GEOLOCALIZACAO

# 1. Valores nulos críticos na tabela dim_geolocalizacao
spark.sql("""
SELECT
  SUM(CASE WHEN cep_prefixo IS NULL THEN 1 END) AS nulo_cep_prefixo,
  SUM(CASE WHEN cidade IS NULL THEN 1 END) AS nulo_cidade,
  SUM(CASE WHEN estado IS NULL THEN 1 END) AS nulo_estado,
  SUM(CASE WHEN latitude IS NULL THEN 1 END) AS nulo_latitude,
  SUM(CASE WHEN longitude IS NULL THEN 1 END) AS nulo_longitude
FROM dim_geolocalizacao;
""").display()

# 2. Verificando valores distintos na tabela dim_geolocalizacao
spark.sql("""
SELECT
  COUNT(*) AS total,
  COUNT(DISTINCT cep_prefixo) AS total_distinct_cep_prefixo,
  COUNT(DISTINCT cidade) AS total_distinct_cidade,
  COUNT(DISTINCT estado) AS total_distinct_estado
FROM dim_geolocalizacao;
""").display()

# 3. Validação do formato do cep_prefixo (5 dígitos) na tabela dim_geolocalizacao
spark.sql("""
SELECT
  COUNT(cep_prefixo) AS total_cep_prefixo,
  COUNT(DISTINCT CASE WHEN LENGTH(cep_prefixo) != 5 THEN cep_prefixo END) AS total_invalidos
FROM dim_geolocalizacao;
""").display()

In [0]:
# QUALIDADE DA TABELA DIM_PAGAMENTOS

# 1. Valores nulos críticos na tabela dim_pagamentos
spark.sql("""
SELECT
  SUM(CASE WHEN id_pagamento IS NULL THEN 1 END) AS nulo_id_pagamento,
  SUM(CASE WHEN tipo_pagamento IS NULL THEN 1 END) AS nulo_tipo_pagamento,
  SUM(CASE WHEN nivel_risco IS NULL THEN 1 END) AS nulo_nivel_risco
FROM dim_pagamentos;
""").display()

# 2. Verificando valores distintos na tabela dim_pagamentos
spark.sql("""
SELECT
  COUNT(*) AS total,
  COUNT(DISTINCT id_pagamento) AS total_distinct_id_pagamento,
  COUNT(DISTINCT tipo_pagamento) AS total_distinct_tipo_pagamento,
  COUNT(DISTINCT nivel_risco) AS total_distinct_nivel_risco
FROM dim_pagamentos;
""").display()

# 3. Verificando duplicidade de id_pagamento
spark.sql("""
SELECT
  id_pagamento,
  COUNT(*) AS qtd
FROM dim_pagamentos
GROUP BY id_pagamento
HAVING COUNT(*) > 1;
""").display()

In [0]:
# QUALIDADE DA TABELA DIM_VENDEDORES

# 1. Valores nulos críticos na tabela dim_vendedores
spark.sql("""
SELECT
  SUM(CASE WHEN vendedor_id IS NULL THEN 1 END) AS nulo_id_vendedor,
  SUM(CASE WHEN cep_prefixo IS NULL THEN 1 END) AS nulo_cep_prefixo
FROM
  dim_vendedores;""").display()

# 2. Validação do formato do cep_prefixo (5 dígitos) na tabela dim_vendedores
spark.sql("""
SELECT
  COUNT(cep_prefixo) AS total_cep_prefixo,
  COUNT(DISTINCT CASE WHEN LENGTH(cep_prefixo) != 5 THEN cep_prefixo END) AS total_invalidos
FROM dim_vendedores;
""").display()

In [0]:
# QUALIDADE DA TABELA FATO_TRANSACOES

# 1. Valores nulos críticos na tabela fato_transacoes
spark.sql("""SELECT
  SUM(CASE WHEN cliente_id IS NULL THEN 1 END) AS nulo_id_cliente,
  SUM(CASE WHEN pedido_id IS NULL THEN 1 END) AS nulo_id_pedido,
  SUM(CASE WHEN vendedor_id IS NULL THEN 1 END) AS nulo_id_vendedor,
  SUM(CASE WHEN data_pedido IS NULL THEN 1 END) AS nulo_data_pedido,
  SUM(CASE WHEN tipo_pagamento IS NULL THEN 1 END) AS nulo_tipo_pagamento,
  SUM(CASE WHEN valor_transacao IS NULL THEN 1 END) AS nulo_valor_transacao
FROM fato_transacoes;""").display()

# 2. Verificando se valor_transacao possui valores negativos
spark.sql("""SELECT
  COUNT(*) AS total_negativos
FROM fato_transacoes
WHERE valor_transacao < 0;""").display()

# 3. Consistência de tipos de dados: checando se data_pedido possui formato válido (YYYY-MM-DD)
spark.sql("""SELECT
  COUNT(*) AS total_invalidos
FROM fato_transacoes
WHERE LENGTH(data_pedido) != 10 OR data_pedido NOT LIKE '____-__-__';""").display()

# 4. Verificando se todos os cliente_id, vendedor_id e tipo_pagamento estão representados nas dimensões
spark.sql("""SELECT
  COUNT(*) AS cliente_nao_representado
FROM fato_transacoes ft
LEFT JOIN dim_clientes dc ON ft.cliente_id = dc.cliente_id
WHERE dc.cliente_id IS NULL;""").display()

spark.sql("""SELECT
  COUNT(*) AS vendedor_nao_representado
FROM fato_transacoes ft
LEFT JOIN dim_vendedores dv ON ft.vendedor_id = dv.vendedor_id
WHERE dv.vendedor_id IS NULL;""").display()

spark.sql("""SELECT
  COUNT(*) AS pagamento_nao_representado
FROM fato_transacoes ft
LEFT JOIN dim_pagamentos dp ON ft.tipo_pagamento = dp.tipo_pagamento
WHERE dp.tipo_pagamento IS NULL;""").display()